# Train Based on clipping

In [1]:
PROJECT = "RFCX"
EXP_NUM = "6"
EXP_TITLE = "Clipping baseline"
EXP_NAME = "exp_" + EXP_NUM + "_" + EXP_TITLE
IS_WRITRE_LOG = False
print('expname:' + EXP_NAME)

expname:exp_6_Clipping baseline


## Library

In [2]:
# library import
import numpy as np
import pandas as pd
import os
import tqdm
import random
import time
import math
import csv
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import logging
import wandb
from time import sleep
from torch.nn import functional as F
from torch.optim import Adam, AdamW
from torchvision.models import resnet18, resnet34, resnet50
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import Subset
import torch.utils.data as torchdata
from torch.optim.lr_scheduler import CosineAnnealingLR, ReduceLROnPlateau
# from transformers import get_linear_schedule_with_warmup
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
from contextlib import contextmanager
from typing import Optional
from numpy.random import beta
from pathlib import Path
from fastprogress.fastprogress import master_bar, progress_bar
from torchviz import make_dot
from conformer import ConformerConvModule
from conformer import ConformerBlock
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit
from torch.utils.tensorboard import SummaryWriter
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)

## Configuration

In [3]:
# use GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [4]:
class dict2(dict): 
    def __init__(self, *args, **kwargs): 
        super().__init__(*args, **kwargs) 
        self.__dict__ = self 

In [9]:
config = dict2({
    "fft":                2048,
    "hop":                512,
    "sr":                 48000,
    "length":             10 * 48000,
    "mel":                384,
    "SEED":               42,
    "INPUT":              Path("../input/rfcx-species-audio-detection/train"),
    "TRAIN_AUDIO_ROOT":   Path("../input/rfcx-species-audio-detection/train_mel"),
    "TEST_AUDIO_ROOT":    Path("../input/rfcx-species-audio-detection/test_mel"),
    "TRAIN_TP":           Path("../input/rfcx-species-audio-detection/train_tp.csv"),
    "TRAIN_TP_MEL":           Path("../input/rfcx-species-audio-detection/train_tp_mel.csv"),
    "SUB":                Path("../input/rfcx-species-audio-detection/sample_submission.csv"),
    # "DIM":                sample.shape[1],
    # "SEQ_LEN":            sample.shape[0],
    "KERNEL_SIZE":        3,
    "KERNEL_STRIDE":      2,
    "KERNEL_SIZE_SEQ":    3,
    "POOL_SIZE":          2,
    "POOL_STRIDE":        2,
    "NUM_BIRDS":          24,
    "N_FOLDS":            5,
    "BATCH_NUM":          100,
    "VALID_BATCH_NUM":    100,
    "EPOCH_NUM":          100,
    "DROPOUT":            0.35,
    "lr":                 0.001,
    "eta_min":            1e-5,
    "t_max":              10,
    "TEST_SIZE":          0.2,
    "MIXUP":              0.0,
    "MIXUP_PROB":         0.0
})

In [10]:
# seed
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore
set_seed(config.SEED)

## Augment

In [11]:
# mixup
def mixup_data(x, y, alpha=1.0, use_cuda=True):

    '''Compute the mixup data. Return mixed inputs, pairs of targets, and lambda'''
    if alpha > 0.:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1.
    batch_size = x.size()[0]
    if use_cuda:
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)
    # lam = max(lam, 1 - lam)
    mixed_x = lam * x + (1 - lam) * x[index,:]
    # mixed_y = lam * y + (1 - lam) * y[index]
    y_a, y_b = y, y[index]
    # return mixed_x, mixed_y
    return mixed_x, y_a, y_b, lam

# def mixup_criterion(y_a, y_b, lam):
#     return lambda criterion, pred: lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

## Transform

In [12]:
# transforms
train_transform = transforms.Compose([
    # transforms.RandomCrop((128, 313), pad_if_needed=True, padding_mode="constant"),
    # transforms.ToTensor()
])
valid_transform = transforms.Compose([
    # transforms.CenterCrop((128, 313)),
    # transforms.ToTensor()
])
label_transform = transforms.Compose([
    # transforms.ToTensor()
])

## Dataset

In [14]:
# Data load
df_train_tp = pd.read_csv(config.TRAIN_TP_MEL)

# add column
for col in range(24):
    df_train_tp[col] = 0

# one-hot encoding
for index, row in df_train_tp.iterrows():
    specId = row["species_id"]
    for col in range(24):
        if int(specId) == col:
            df_train_tp.iloc[index, df_train_tp.columns.get_loc(col)] = 1

# grouping
# df_train_tp = df_train_tp.groupby("recording_id", as_index=False).max()

# check
print(len(df_train_tp))
display(df_train_tp[df_train_tp["recording_id"] == "77299bde7"].head())


1216


,id,species_id,recording_id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
584,584,8,77299bde7,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
585,585,21,77299bde7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [20]:
# load data
ids = []
specIds = []
record_ids = []
labels = []
for index, row in df_train_tp.iterrows():
    ids.append(row.values[0])
    specIds.append(row.values[1])
    record_ids.append(row.values[2])
    labels.append(row.values[3:27])

labels = np.array(labels).astype(float)

print(ids[584])
print(specIds[584])
print(record_ids[584])
print(labels[584])
print(labels[584].shape)

584
8
77299bde7
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
(24,)
